In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(qs)
})

In [2]:
source("../Utility/gene_symbol_conversion.r")
source("../Utility/Matrix.utils.R")

In [3]:
View(update_gene_names)

function (seurat_obj) 
{
    genes <- rownames(seurat_obj)
    if (!any(grepl("^ENSG", genes))) {
        message("Gene names already appear to be symbols. No conversion needed.")
        return(seurat_obj)
    }
    message("Converting Ensembl IDs to gene symbols...")
    gene_symbols <- convert_ensembl_to_symbol(genes)
    valid <- !is.na(gene_symbols) & gene_symbols != ""
    if (sum(valid) == 0) {
        warning("No valid gene symbols found. Returning original object.")
        return(seurat_obj)
    }
    seurat_obj <- seurat_obj[valid, ]
    new_names <- gene_symbols[valid]
    if (any(duplicated(new_names))) {
        counts <- GetAssayData(seurat_obj, slot = "counts")
        counts_agg <- aggregate.Matrix(counts, groupings = new_names, 
            fun = "sum")
        seurat_obj <- CreateSeuratObject(counts = counts_agg, 
            meta.data = seurat_obj@meta.data, min.cells = 3, 
            min.features = 200)
    }
    else {
        rownames(seurat_obj) <- new_names
    }
    seurat_obj[["nFeature_RNA"]] <- apply(GetAssayData(seurat_obj, 
        slot = "counts"), 2, function(x) sum(x > 0))
    return(seurat_obj)
}

In [4]:
View(convert_ensembl_to_symbol)

function (ensembl_ids) 
{
    ensembl_ids_clean <- sub("\\..*", "", ensembl_ids)
    mapIds(org.Hs.eg.db, keys = ensembl_ids_clean, column = "SYMBOL", 
        keytype = "ENSEMBL", multiVals = "first")
}

In [6]:
seurat_list <- qread("./out/seurat_list_doublets_SCPCP000001.qs")

In [7]:
for (sample in names(seurat_list)) {
    seurat_list[[sample]] <- update_gene_names(seurat_list[[sample]])
}


Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”
Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to gene symbols...

'select()' returned 1:many mapping between keys and columns

Converting Ensembl IDs to 

In [8]:
for (sample_name in names(seurat_list)) {
    subset_obj <- seurat_list[[sample_name]]
    subset_obj <- subset(subset_obj, subset=(DFC_1=="Singlet"))
    subset_obj[["percent.mt"]] <- PercentageFeatureSet(object = subset_obj, pattern = "^MT-")
    seurat_list[[sample_name]] <- subset(subset_obj, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & percent.mt < 20)
}

In [9]:
qsave(seurat_list, "./out/seurat_list_filtered_SCPCP00000.qs")

In [10]:
seurat_list

$SCPCL000002
An object of class Seurat 
18307 features across 864 samples within 1 assay 
Active assay: RNA (18307 features, 0 variable features)
 1 layer present: counts

$SCPCL000003
An object of class Seurat 
8314 features across 246 samples within 1 assay 
Active assay: RNA (8314 features, 0 variable features)
 1 layer present: counts

$SCPCL000004
An object of class Seurat 
18294 features across 331 samples within 1 assay 
Active assay: RNA (18294 features, 0 variable features)
 1 layer present: counts

$SCPCL000006
An object of class Seurat 
18547 features across 1227 samples within 1 assay 
Active assay: RNA (18547 features, 0 variable features)
 1 layer present: counts

$SCPCL000007
An object of class Seurat 
17808 features across 968 samples within 1 assay 
Active assay: RNA (17808 features, 0 variable features)
 1 layer present: counts

$SCPCL000008
An object of class Seurat 
17842 features across 3472 samples within 1 assay 
Active assay: RNA (17842 features, 0 variable feat